# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 22 2023 11:08AM,263387,15,9504439,"Brookfield Pharmaceuticals, LLC",Released
1,May 22 2023 11:08AM,263387,15,9504449,"Brookfield Pharmaceuticals, LLC",Released
2,May 22 2023 11:06AM,263386,19,9504416,Purify Life,Released
3,May 22 2023 10:58AM,263385,15,9504429,"Brookfield Pharmaceuticals, LLC",Released
4,May 22 2023 10:58AM,263385,15,9504431,"Brookfield Pharmaceuticals, LLC",Released
5,May 22 2023 10:58AM,263385,15,9504433,"Brookfield Pharmaceuticals, LLC",Released
6,May 22 2023 10:58AM,263385,15,9504434,"Brookfield Pharmaceuticals, LLC",Released
7,May 22 2023 10:58AM,263385,15,9504435,"Brookfield Pharmaceuticals, LLC",Released
8,May 22 2023 10:58AM,263385,15,9504442,"Brookfield Pharmaceuticals, LLC",Released
9,May 22 2023 10:58AM,263385,15,9504443,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
21,263384,Executing,2
22,263384,Released,1
23,263385,Released,16
24,263386,Released,1
25,263387,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
263383,NaN,16.0
263384,2.0,1.0
263385,NaN,16.0
263386,NaN,1.0
263387,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
263351,1.0,0.0
263361,0.0,20.0
263362,0.0,4.0
263363,0.0,1.0
263365,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
263351,1,0
263361,0,20
263362,0,4
263363,0,1
263365,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,263351,1,0
1,263361,0,20
2,263362,0,4
3,263363,0,1
4,263365,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,263351,1,
1,263361,,20
2,263362,,4
3,263363,,1
4,263365,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC"
2,May 22 2023 11:06AM,263386,19,Purify Life
3,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC"
19,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd"
22,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC"
38,May 22 2023 10:48AM,263382,10,Eye Pharma Inc
39,May 22 2023 10:47AM,263381,20,"HVL, LLC dba Atrium Innovations"
40,May 22 2023 10:44AM,263380,19,"GCH Granules USA, Inc."
41,May 22 2023 10:43AM,263379,10,"CLINUVEL, Inc."
42,May 22 2023 10:39AM,263378,10,MedStone Pharma LLC


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",,2
1,May 22 2023 11:06AM,263386,19,Purify Life,,1
2,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC",,16
3,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd",2,1
4,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC",,16
5,May 22 2023 10:48AM,263382,10,Eye Pharma Inc,,1
6,May 22 2023 10:47AM,263381,20,"HVL, LLC dba Atrium Innovations",1,
7,May 22 2023 10:44AM,263380,19,"GCH Granules USA, Inc.",,1
8,May 22 2023 10:43AM,263379,10,"CLINUVEL, Inc.",,1
9,May 22 2023 10:39AM,263378,10,MedStone Pharma LLC,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",2,
1,May 22 2023 11:06AM,263386,19,Purify Life,1,
2,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC",16,
3,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd",1,2
4,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC",16,
5,May 22 2023 10:48AM,263382,10,Eye Pharma Inc,1,
6,May 22 2023 10:47AM,263381,20,"HVL, LLC dba Atrium Innovations",,1
7,May 22 2023 10:44AM,263380,19,"GCH Granules USA, Inc.",1,
8,May 22 2023 10:43AM,263379,10,"CLINUVEL, Inc.",1,
9,May 22 2023 10:39AM,263378,10,MedStone Pharma LLC,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",2,
1,May 22 2023 11:06AM,263386,19,Purify Life,1,
2,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC",16,
3,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd",1,2
4,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC",16,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",2.0,NaN
1,May 22 2023 11:06AM,263386,19,Purify Life,1.0,NaN
2,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC",16.0,NaN
3,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd",1.0,2.0
4,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC",16.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,May 22 2023 11:08AM,263387,15,"Brookfield Pharmaceuticals, LLC",2.0,0.0
1,May 22 2023 11:06AM,263386,19,Purify Life,1.0,0.0
2,May 22 2023 10:58AM,263385,15,"Brookfield Pharmaceuticals, LLC",16.0,0.0
3,May 22 2023 10:57AM,263384,20,"AdvaGen Pharma, Ltd",1.0,2.0
4,May 22 2023 10:55AM,263383,15,"Brookfield Pharmaceuticals, LLC",16.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2897086,63.0,60.0
15,1053529,110.0,8.0
16,263363,1.0,0.0
19,526766,2.0,0.0
20,526765,1.0,3.0
21,263376,1.0,0.0
22,526720,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2897086,63.0,60.0
1,15,1053529,110.0,8.0
2,16,263363,1.0,0.0
3,19,526766,2.0,0.0
4,20,526765,1.0,3.0
5,21,263376,1.0,0.0
6,22,526720,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,63.0,60.0
1,15,110.0,8.0
2,16,1.0,0.0
3,19,2.0,0.0
4,20,1.0,3.0
5,21,1.0,0.0
6,22,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,63.0
1,15,Released,110.0
2,16,Released,1.0
3,19,Released,2.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Executing,60.0,8.0,0.0,0.0,3.0,0.0,1.0
Released,63.0,110.0,1.0,2.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Executing,60.0,8.0,0.0,0.0,3.0,0.0,1.0
1,Released,63.0,110.0,1.0,2.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Executing,60.0,8.0,0.0,0.0,3.0,0.0,1.0
1,Released,63.0,110.0,1.0,2.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()